In [107]:
from __future__ import print_function
import os
import json
import re
# import chardet
import argparse
import pandas as pd
import google.auth
from google.oauth2 import service_account
from google.cloud import storage
from google.cloud import bigquery
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

dev_key_path = "/Users/gowid/Desktop/master/transaction_analysis/config/gowid-gcs-dev_keyfile.json"
prd_key_path = "/Users/gowid/Desktop/master/transaction_analysis/config/gowid-gcs-prd-f069a99a7d10.json"

# sql_path = '/Users/gowid/Desktop/master/cashflow_analysis/sql/'

dev_credentials = service_account.Credentials.from_service_account_file(dev_key_path,scopes=['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/bigquery','https://www.googleapis.com/auth/devstorage.full_control'])
prd_credentials = service_account.Credentials.from_service_account_file(prd_key_path,scopes=['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/bigquery','https://www.googleapis.com/auth/devstorage.full_control'])

#client 설정
client = storage.Client(credentials = dev_credentials, project = dev_credentials.project_id)
bq_client = bigquery.Client(credentials = dev_credentials, project = dev_credentials.project_id)

# 연동할 google drive folder id
google_drive_id = '1cO7bI_2opLbwrDi5tN7kWEqrxMPwWh65'

In [102]:
from google.cloud import storage
idxcorp = '1'
bucket_name = "crm_bucket_test"

# bucket list 확인
bucket_list = client.list_buckets()

print("Buckets in {}:".format(client.project))
for item in bucket_list:
    print("\t" + item.name)

# 대상폴더명 리스트
sql = f"""
SELECT cast(idxCorp as string) as idxCorp
     , appliedAt
     , issuanceStatus
FROM `gowid-gcs-prd.raw_stream_gowid.CardIssuanceInfo`
WHERE issuanceStatus in ('APPLY','ISSUED')
      AND cardType in ('GOWID')
      AND PARSE_DATE('%Y%m%d',FORMAT_DATETIME('%Y%m%d',appliedAt)) <= CURRENT_DATE()
ORDER BY idxCorp
"""

# 데이터 조회 쿼리 실행 결과
query_job = bq_client.query(sql)

# 데이터프레임 변환
# type(query_job)
df = query_job.to_dataframe()
folder = df['idxCorp']
folder_list = folder.to_list()
# print(folder_list)
print(len(folder_list))

Buckets in gowid-gcs-dev:
	asia.artifacts.gowid-gcs-dev.appspot.com
	crm_bucket_test
	crm_bucket_test_2
	gcf-sources-825455146653-asia-northeast3
	gcf-sources-825455146653-us-central1
	gowid-gcs-dev-ocr
	gowid_gcs_dev_shareholders
	ocr_list
	us.artifacts.gowid-gcs-dev.appspot.com
2212


In [80]:
# idxcorp = 'ttt'
# current_bucket = client.get_bucket("crm_bucket_test")
# folder_path = current_bucket.name+'/'+idxcorp
# print(current_bucket.name+'/'+idxcorp)
# blob = current_bucket.blob(idxcorp+'/')
# print(blob.path)
# print(blob.name)

crm_bucket_test/ttt
/b/crm_bucket_test/o/ttt%2F
ttt/


In [135]:
# idxcorp 를 변수로 받아서 bucket 내 folder 를 만들기
corp_id = ['고위드','포잉','프레시코드']
def create_folder(bucket_name, destination_folder_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(destination_folder_name)

    blob.upload_from_string('')

    blob_path.append(blob.path)
    blob_name.append(blob.name)

    print('Created {} . bucket name : {} / path : {}'.format(
        destination_folder_name, blob_path, blob_name))

# create_folder("crm_bucket_test", 'test-folder/')

# Create folder in bucket 
blob_path = []
blob_name = []
for idxcorp in corp_id: 
    destination_folder_name = idxcorp+'/'
    # if legacy folder exist , replace(delete & create)  logic 은 bucket은 이름이 같으면 중복으로 안생기네
    create_folder("crm_bucket_test_3", destination_folder_name)
    # print(destination_folder_name)
    print(blob_path)
    print(blob_name)
# print(bucket)



Created 고위드/ . bucket name : ['/b/crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F'] / path : ['고위드/']
['/b/crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F']
['고위드/']
Created 포잉/ . bucket name : ['/b/crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F', '/b/crm_bucket_test_3/o/%ED%8F%AC%EC%9E%89%2F'] / path : ['고위드/', '포잉/']
['/b/crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F', '/b/crm_bucket_test_3/o/%ED%8F%AC%EC%9E%89%2F']
['고위드/', '포잉/']
Created 프레시코드/ . bucket name : ['/b/crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F', '/b/crm_bucket_test_3/o/%ED%8F%AC%EC%9E%89%2F', '/b/crm_bucket_test_3/o/%ED%94%84%EB%A0%88%EC%8B%9C%EC%BD%94%EB%93%9C%2F'] / path : ['고위드/', '포잉/', '프레시코드/']
['/b/crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F', '/b/crm_bucket_test_3/o/%ED%8F%AC%EC%9E%89%2F', '/b/crm_bucket_test_3/o/%ED%94%84%EB%A0%88%EC%8B%9C%EC%BD%94%EB%93%9C%2F']
['고위드/', '포잉/', '프레시코드/']


In [ ]:
#bucket의 내용을 임시 디렉토리에 받는다 (1회성)
# 임시 디렉토리에 받은 내용을 다시 드라이브에 올린다. 
# bucket에서 받은 내용을 경로단위(?) 로 정확히 drive의 경로로 만들어서 올려야 하는데 
# 해야할일 : bucket에서 파일이 포함 된 folder sample / file sample을 만든다. 
# folder sample의 경로 / uri 뭐든 이어받을 수 있는 내용 확인한다. 
# bucket 의 경로 -> tmp dir 의 경로 -> 구글드라이브 upload 경로 과정에서 파싱되어야 하는 경로들을 하나하나 변수화 한다. 
# 현재 함수는 파일 1개 업로드 하는건 동기화 되어 있음 (되어 있는 기능 : "연동")
# 부족한 기능은 경로 단위로 상위 디렉토리(폴더까지 함께 동기화 되도록 만들어야 함 
# 


In [162]:
bucket = client.get_bucket("crm_bucket_test_3")

bucket_name = bucket.name
list_blob_b = bucket.list_blobs()
blob_list = []
folder_path_list = []
for blob in list_blob_b:
    print(blob.name)
    blob_name = blob.name
    link = blob.path_helper(bucket_name, blob_name)
    folder_path = "/b/" + bucket_name + "/o/"+ blob_name
    print(folder_path)
    print('gs://'+link)
    blob_list.append(blob_name)
    folder_path_list.append(folder_path)
print(blob_list)
print(folder_path_list)

고위드/
/b/crm_bucket_test_3/o/고위드/
gs://crm_bucket_test_3/o/%EA%B3%A0%EC%9C%84%EB%93%9C%2F
포잉/
/b/crm_bucket_test_3/o/포잉/
gs://crm_bucket_test_3/o/%ED%8F%AC%EC%9E%89%2F
프레시코드/
/b/crm_bucket_test_3/o/프레시코드/
gs://crm_bucket_test_3/o/%ED%94%84%EB%A0%88%EC%8B%9C%EC%BD%94%EB%93%9C%2F
['고위드/', '포잉/', '프레시코드/']
['/b/crm_bucket_test_3/o/고위드/', '/b/crm_bucket_test_3/o/포잉/', '/b/crm_bucket_test_3/o/프레시코드/']


In [163]:
bucket = client.get_bucket("crm_bucket_test_3")
# for blob_list in bucket.list_blobs('고위드/'):
for key in folder_path_list:
    blob = bucket.blob(key)
    tmpdir = "/tmp/file"
    blob_downloaded = blob.download_to_filename(tmpdir)
    print(blob_downloaded)


NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/crm_bucket_test_3/o/%2Fb%2Fcrm_bucket_test_3%2Fo%2F%EA%B3%A0%EC%9C%84%EB%93%9C%2F?alt=media: No such object: crm_bucket_test_3//b/crm_bucket_test_3/o/고위드/: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [ ]:

def download_file_from_gcs(bucket, key):
    print("Downloading key: {} in bucket: {}".format(key, bucket))
    source_bucket = client.bucket(bucket)
    blob_object = source_bucket.blob(key)
    tmpdir = "/tmp/file"
    blob_object.download_to_filename(tmpdir)
    return tmpdir

In [99]:
bucket = "crm_bucket_test_2"

In [132]:
# create folder in google drive test
### bucket 이랑 연동하려면 blob_name을 받아서 blob name과 동일한 폴더를 만들어야함 

from googleapiclient import discovery

print(blob_list)
google_drive_id = '1cO7bI_2opLbwrDi5tN7kWEqrxMPwWh65'
dev_credentials = service_account.Credentials.from_service_account_file(dev_key_path,scopes=['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/bigquery','https://www.googleapis.com/auth/devstorage.full_control'])
service = discovery.build('drive', 'v3', dev_credentials)

def create_folder():
    # google_drive_id = '1cO7bI_2opLbwrDi5tN7kWEqrxMPwWh65'
    # dev_credentials = service_account.Credentials.from_service_account_file(dev_key_path,scopes=['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/bigquery','https://www.googleapis.com/auth/devstorage.full_control'])
    # service = discovery.build('drive', 'v3', credentials=dev_credentials)
    try:
        for name_list in blob_list:
            file_metadata = {
                    'name': name_list,
                    'mimeType': 'application/vnd.google-apps.folder'
                    ,'parents' : [google_drive_id]
                }
            # folder = drive.Create 
            file = service.files().create(supportsAllDrives=True, body=file_metadata, fields = "id").execute()
            # print(F'{file.key()}')
            print(F'Folder ID: "{file.get("id")}".')
        return file.get('id')

    except HttpError as error:
        print(F'An error occurred: {error}')
        return None        

if __name__ == '__main__':
    test_list = ['1','2','3']
    create_folder()

Folder ID: "1ZLQPlwR-An4WNZV0_irlNRtDlO8Yoagv".
Folder ID: "1zSj4E3uH4COCiUPHSU6E3Pif4wRAURVu".
Folder ID: "1c7dSydl923iVATjc7sP_3aJBx8o5gUWa".


In [106]:
import random
import json
import os
import string
import traceback
from googleapiclient import discovery
import google.auth
from googleapiclient.http import MediaFileUpload
from google.cloud import storage

dev_key_path = "/Users/gowid/Desktop/master/transaction_analysis/config/gowid-gcs-dev_keyfile.json"
dev_credentials = service_account.Credentials.from_service_account_file(dev_key_path,scopes=['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/bigquery','https://www.googleapis.com/auth/devstorage.full_control'])
client = storage.Client(credentials = dev_credentials, project = dev_credentials.project_id)

def upload_to_drive(local_path, remote_path):
    GDRIVE_FOLDER_IDS = str(os.environ["GDRIVE_FOLDER_IDS"]).split(",")
    credentials, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/drive"])
    service = discovery.build('drive', 'v3', credentials=credentials)
    folder_metadata = {
            'name': str(local_path),
            'mimeType': 'application/vnd.google-apps.folder',
            'parents' : str(os.environ["GDRIVE_FOLDER_IDS"]).split(",")
        }
    folder_list = []
    folder = service.files().create(body=folder_metadata, fields='id'
                                      ).execute()
    for i in folder.get('id'):
        folder_list.append(folder.get('id'))
    # file 을 올리는것만되어 있고, 
    # folder 올리는 것은 처리가 안되어 있음 
    
    media = MediaFileUpload(local_path)
    print("Starting to upload in the drive")
    response = service.files().create(media_body=media, body={"name": remote_path.split("/")[-1], "parents": GDRIVE_FOLDER_IDS}).execute()

    print(response)

def download_file_from_gcs(bucket, key):
    print("Downloading key: {} in bucket: {}".format(key, bucket))
    client = storage.Client(credentials = dev_credentials, project = dev_credentials.project_id)
    source_bucket = client.bucket(bucket)
    blob_object = source_bucket.blob(key)
    tmpdir = "/tmp/file"
    blob_object.download_to_filename(tmpdir)
    return tmpdir

def drive_upload(event, context):
    try:
        if event["name"][-1] == "/":
            print("Folder: {} is created in the bucket: {}".format(
                event["name"],
                event["bucket"]
            ))
            return

        # Downloading file from GCS
        local_path = download_file_from_gcs(event["bucket"], event["name"])

        # Uploading the file to drive
        upload_to_drive(local_path, event["name"])

    except Exception as e:
        print(traceback.format_exc())

In [53]:
import random
import json
import os
import string
import traceback
from googleapiclient import discovery
import google.auth
from googleapiclient.http import MediaFileUpload
from google.cloud import storage

google_drive_id = "1cO7bI_2opLbwrDi5tN7kWEqrxMPwWh65"
credentials = google.auth.default(scopes=["https://www.googleapis.com/auth/drive"])
service = discovery.build('drive','v3',credentials = dev_credentials)
media = MediaFileUpload(local_path)
# GDRIVE_FOLDER_IDS = str(os.environ['google_drive_id']).split(",")

['None']
